In [ ]:
from nitropulse import RismaData

In [ ]:
risma = RismaData(workspace_dir='/home/morteza/.nitropulse')

In [ ]:
risma_stations = ['RISMA_MB1', 'RISMA_MB2', 
                  'RISMA_MB3', 'RISMA_MB4', 
                  'RISMA_MB5', 'RISMA_MB6', 
                  'RISMA_MB7', 'RISMA_MB8', 
                  'RISMA_MB9', 'RISMA_MB10', 
                  'RISMA_MB11', 'RISMA_MB12', 
                  'RISMA_MB13', 'RISMA_MB14', 
                  'RISMA_MB15', 'RISMA_MB16',
                  'RISMA_MB17', 'RISMA_MB18', 
                  'RISMA_MB19', 'RISMA_MB20',
                  'RISMA_MB21', 'RISMA_MB22', 
                  'RISMA_MB23', 'RISMA_MB24',
                  'RISMA_MB25', 'RISMA_MB26',
                  'RISMA_SK1', 'RISMA_SK2', 
                  'RISMA_SK3', 'RISMA_SK4',

                  ]
params = ['Air Temp', 'Soil temperature', 'Soil Moisture']
sensors= 'average'
depths = ['0 to 5 cm', '5 cm']

In [ ]:
# risma.download_risma_data(
#     out_dir='../assets/inputs/RISMA_CSV_files', 
#     stations=risma_stations, parameters=params, 
#     sensors=sensors, depths=depths, 
#     start_date='2010-01-01', end_date='2024-01-01')

In [ ]:
risma_df = risma.load_df(depth='0 to 5 cm')
print(risma_df.shape)
risma_df.head()

In [ ]:
# risma_df.to_csv("/home/morteza/.nitropulse/outputs/risma_df.csv", index=False)

In [1]:
import pandas as pd

risma_df = pd.read_csv('~/.nitropulse/outputs/risma_df.csv')
risma_df.head()

,date,station,depth,mean_sst,mean_airt,airt,prcp,ssm,sst,year,doy,depth_cm,bulk,sand,silt,clay,classification
0,2013-04-16 18:00:00,MB1,0 to 5 cm,0.108333,-1.52,-1.82,0.0,-0.160000,0.000000,2013,106,5,1.28,0.788,0.101,0.111,Sandy Loam
1,2013-04-16 18:15:00,MB1,0 to 5 cm,0.108333,-1.52,-2.02,0.0,0.206667,0.050000,2013,106,5,1.28,0.788,0.101,0.111,Sandy Loam
2,2013-04-16 18:30:00,MB1,0 to 5 cm,0.108333,-1.52,-1.89,0.0,0.206667,0.050000,2013,106,5,1.28,0.788,0.101,0.111,Sandy Loam
3,2013-04-16 18:45:00,MB1,0 to 5 cm,0.108333,-1.52,-1.67,0.0,0.208333,0.083333,2013,106,5,1.28,0.788,0.101,0.111,Sandy Loam
4,2013-04-16 19:00:00,MB1,0 to 5 cm,0.108333,-1.52,-1.72,0.0,0.208333,0.083333,2013,106,5,1.28,0.788,0.101,0.111,Sandy Loam


In [2]:
# filter data for a specific station and year
year = 2023
station_id = 'MB3'

# count records for station MB15 (matches MB15, MB_15 or MB-15) in year 2015
mask_station = risma_df['station'].astype(str).str.contains(station_id, case=False, na=False)
mask_year = risma_df['year'] == year
sub_df = risma_df.loc[mask_station & mask_year]
sub_df.head()

,date,station,depth,mean_sst,mean_airt,airt,prcp,ssm,sst,year,doy,depth_cm,bulk,sand,silt,clay,classification
6597056,2023-01-01 00:00:00,MB3,0 to 5 cm,-4.875,-8.315,-8.30,0.0,0.116667,-5.016667,2023,1,5,1.47,0.471,0.211,0.318,Sandy Clay Loam
6597163,2023-01-01 01:15:00,MB3,0 to 5 cm,-4.875,-8.315,-8.68,0.0,0.116667,-5.033333,2023,1,5,1.47,0.471,0.211,0.318,Sandy Clay Loam
6597194,2023-01-01 01:30:00,MB3,0 to 5 cm,-4.875,-8.315,-8.72,0.0,0.116667,-5.050000,2023,1,5,1.47,0.471,0.211,0.318,Sandy Clay Loam
6597225,2023-01-01 01:45:00,MB3,0 to 5 cm,-4.875,-8.315,-8.78,0.0,0.116667,-5.050000,2023,1,5,1.47,0.471,0.211,0.318,Sandy Clay Loam
6597256,2023-01-01 02:00:00,MB3,0 to 5 cm,-4.875,-8.315,-8.86,0.0,0.115000,-5.083333,2023,1,5,1.47,0.471,0.211,0.318,Sandy Clay Loam


In [3]:
# aggregate to daily (sum precipitation, mean SSM) so they can be plotted together
sub_df_day = sub_df.copy()
sub_df_day["date"] = pd.to_datetime(sub_df_day["date"]).dt.floor("D")

precip_daily = (
    sub_df_day.groupby("date", as_index=False)["prcp"]
    .sum()
    .rename(columns={"prcp": "prcp_daily"})
)
sst_daily = (
    sub_df_day.groupby("date", as_index=False)["sst"]
    .mean()
    .rename(columns={"sst": "sst_daily"})
)

# merged table for easy plotting/inspection
daily = pd.merge(sst_daily, precip_daily, on="date", how="outer").sort_values("date").reset_index(drop=True)
daily["prcp_daily"] = daily["prcp_daily"].fillna(0)
daily["station"] = "MB15"

daily.head()

,date,sst_daily,prcp_daily,station
0,2023-01-01,-4.967210,0.0,MB15
1,2023-01-02,-5.436458,0.0,MB15
2,2023-01-03,-7.083333,0.0,MB15
3,2023-01-04,-7.775521,0.0,MB15
4,2023-01-05,-8.280729,0.0,MB15


In [4]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- Prep dates ---
sub_df = sub_df.copy()
daily = daily.copy()
sst_daily = sst_daily.copy()

sub_df["date"]    = pd.to_datetime(sub_df["date"]).dt.floor("D")
daily["date"]     = pd.to_datetime(daily["date"]).dt.floor("D")
sst_daily["date"] = pd.to_datetime(sst_daily["date"]).dt.floor("D")

# --- Find contiguous frozen ranges (sst ≤ 0 °C) ---
sst_daily["is_frozen"] = sst_daily["sst_daily"] <= 0
grp = (sst_daily["is_frozen"] != sst_daily["is_frozen"].shift()).cumsum()
frozen_ranges = (
    sst_daily.loc[sst_daily["is_frozen"]]
    .groupby(grp)
    .agg(start=("date", "min"), end=("date", "max"))
    .reset_index(drop=True)
)

# --- Build figure with secondary y-axis ---
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 1) Frozen shading (background)
for _, r in frozen_ranges.iterrows():
    fig.add_shape(
        type="rect", xref="x", yref="paper",
        x0=r["start"], x1=r["end"], y0=0, y1=1,
        fillcolor="LightSkyBlue", opacity=0.25,
        layer="below", line_width=0
    )

# 2) Soil moisture lines
for st, df_st in sub_df.sort_values("date").groupby("station"):
    fig.add_trace(
        go.Scatter(
            x=df_st["date"], y=df_st["ssm"],
            name=st, mode="lines", line=dict(width=2),
            connectgaps=False
        ),
        secondary_y=False
    )

# 3) Precipitation bars
one_day_ms = 24 * 60 * 60 * 1000
fig.add_trace(
    go.Bar(
        x=daily["date"], y=daily["prcp_daily"],
        name="Precipitation", marker_color="red",
        opacity=0.6, marker_line=dict(width=0),
        width=0.9 * one_day_ms
    ),
    secondary_y=True
)

# Layout & axes with bigger fonts
fig.update_layout(
    showlegend=True,
    legend_title_text="Stations",
    font=dict(family="Arial", size=14, color="Black"),  # base font
    bargap=0,
    barmode="overlay",
    annotations=[dict(
        xref="paper", x=0.01, y=0.4, xanchor="left", yanchor="top",
        text="Shaded = frozen soil (sst ≤ 0 °C)",
        showarrow=False, bgcolor="rgba(255,255,255,0.7)",
        bordercolor="LightSkyBlue",
        font=dict(size=12)
    )],
    width=1500,
    height=500,
)

fig.update_xaxes(
    title_text="Date",
    title_font=dict(size=18),
    tickfont=dict(size=14)
)
fig.update_yaxes(
    title_text="Soil Moisture (m³/m³)",
    secondary_y=False,
    title_font=dict(size=18),
    tickfont=dict(size=14)
)
fig.update_yaxes(
    title_text="Precipitation (mm/day)",
    secondary_y=True,
    title_font=dict(size=18),
    tickfont=dict(size=14)
)

fig.show()
